## Парсинг и анализ данных

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("/home/urev/projects/master-medicine/")
    
import random
import re
import time
import json
import datetime as dt
import dataclasses

import requests
import typing as t
import yaml

from pprint import pprint

from pathlib import Path

import pandas as pd
from tqdm.auto import tqdm

DIR_HOME = str(Path.home())

import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

In [2]:
from docx.table import Table
from docx.text.paragraph import Paragraph
from docx.api import Document

from parser.main import save

In [28]:
@dataclasses.dataclass
class ParseResult:
    folder_name: str
    dfs: t.List[t.List[pd.DataFrame]]
    skipped_dfs: t.List

In [4]:
def create_dataframe_from_rows(table) -> pd.DataFrame:
    data = []
    
    keys = None
    for i, row in enumerate(table.rows):
        text = (cell.text for cell in row.cells)
    
        if i == 0:
            keys = tuple(text)
            continue
        row_data = dict(zip(keys, text))
        data.append(row_data)

    return pd.DataFrame(data)

def processing_name(name: str) -> str:
    """Подпись перед таблицей может быть на нескольких строках, пока дропнем всё кроме первой строки"""
    # return name.lower().split("\n")[0].strip()
    return name.lower().replace("\n", " ")

def processing_folder_name(raw_folder_name: str) -> str:
    folder_name = re.findall(r"\d\.\s+(.*)", raw_folder_name)
    if not folder_name:
        return raw_folder_name
    else:
        return folder_name[-1]

In [5]:
def get_docx(path: str) -> t.List[str]:
    for f in sorted(os.listdir(path)):
        if not "R" in f or not f.endswith("docx"):
            continue
        yield os.path.join(path, f)

In [49]:
[*get_docx("../raw/2021/")]

In [29]:
docx_file = "/home/urev/projects/master-medicine/raw/2013/R-1.docx"

def get_dfs_from_docx(docx_file: str) -> ParseResult:
    """
    Таблицы бывают двух видов:
    - узкие с продолжением в высоту (у таких подписано 'Продолжение табл. X.X') - эти автоматически склеятся
    - широкие с продолжением в ширину (такие автоматически не парсятся, их надо клеить). Чтобы корректно склеить необходимо добавить разметку в файл:
     - Поправить наименование таблицы, тк скорее всего оно тоже широкое и разделено на две части
     - таблицы именуются по имени главной, например, 2.76 + следующая нумерация через точку -> 2.76.1
     - новая таблица (фактически часть по высоте) обозначается как 2.76.1
     - её вертикальное продолжение помечается как ВЕРТИКАЛЬНОЕ продолжение табл. 2.76.1

    При такой разметке: узкие таблицы сами склеятся в одну общую, широкие таблицы склеятся по ширине, 
    по высоте надо будет клеить после (2.76 + 2.76.1 + 2.76.2 и тд)
    """
    document = Document(docx_file)
    
    dfs = {}
    last_obj_stack = []
    last_table_obj = None
    raw_folder_name = None
    skipped_dfs = {}
    
    for idx, c in enumerate(document.sections[0].iter_inner_content()):
        if isinstance(c, Paragraph) and idx == 0:
            raw_folder_name = c.text
            print(f"raw_folder_name: {raw_folder_name}")
        if isinstance(c, Paragraph):
            # print(f"--> {c.text}")
            pass
        if isinstance(c, Table):
            tmp = create_dataframe_from_rows(table=c)
    
            last_obj = last_obj_stack.pop()
            name = last_obj.text.lower()
            print(f"--> Таблица: {name}")
    
            pattern_name = "\d\.\d+"

            if "пропуск" in name:
                print(f"SKIP, обнаружена разметка пропуска: {name}")
                tmp.name = name

                skipped_dfs[tmp.name] = tmp
                
                display(tmp.head())
                continue
            
            if re.findall(pattern_name, name):
                table_number = re.findall(pattern_name, name)[-1]
                print(f"✅ В имени таблицы (объект перед таблицей) есть цифры: {table_number} -> это или исходная таблица или продолжение!")
            else:
                print(f"🔴 В имени таблицы (объект перед таблицей) нет цифр, попытка взять еще один объект из стека (может быть имя на нескольких строках)")
                if last_obj_stack != []:
                    # если стек не пустой
                    last_obj = last_obj_stack.pop()

                    if isinstance(last_obj, Table):
                        print(f"👀Предыдущий объект = таблица, СКОРЕЕ ВСЕГО НУЖНА РАЗМЕТКА!")
                    elif isinstance(last_obj, Paragraph):
                        print(f"👀 Предыдущий объект = параграф с текстом, извлекаем имя....")
                    
                        name = last_obj.text.lower()
                        if not re.findall(pattern_name, name):
                            print(f"⛔️ В предыдущем объекте, так же нет цифр, эта таблица нам не подходит! name: {name}")
                            continue
                        else:
                            print(f"✅ Имя таблицы извлечено: {name}")
                else:
                    name = "unknown"
                    print("⛔️ Стек пустой!")
                print(f"☝️ В предыдущем obj нет цифр, берем следующий из стека (или 'unknown'), имя таблицы: {name}")
                
            tmp.name = processing_name(name)
            
            if getattr(last_obj, "text") and "продолжение" in last_obj.text.lower() and "вертикальное" not in last_obj.text.lower():
                print(f"\t--> Продолжение таблицы: {last_table_obj.name}")
                tmp = pd.concat([last_table_obj, tmp])
                tmp.name = last_table_obj.name
                display(tmp.sample(5))
            elif getattr(last_obj, "text") and "вертикальное" in last_obj.text.lower():
                print(f"🔫 Это вертикальное продолжение таблицы: {last_table_obj.name}")
                tmp = pd.concat([last_table_obj, tmp], axis="columns")
                tmp.name = last_table_obj.name
                display(tmp.sample(5))
            else:
                display(tmp.head())
    
            dfs[tmp.name] = tmp
            
            last_table_obj = tmp
    
        last_obj_stack.append(c)

    return ParseResult(
        folder_name=processing_folder_name(raw_folder_name),
        dfs=[[df] for df in dfs.values()],
        skipped_dfs=[[df] for df in skipped_dfs.values()]
    )

In [41]:
_parse_obj = get_dfs_from_docx(docx_file="../raw/2015/R-1.docx")

In [42]:
_parse_obj.folder_name

In [53]:
year = 2021
for doc_path in get_docx(f"../raw/{year}/"):
    if re.match(r'.*r-[8-9].*', doc_path.lower()):
        continue
    print(doc_path)
    parse_obj = get_dfs_from_docx(docx_file=doc_path)

    folder_name = doc_path.split("/")[-1].split(".")[0].lower() if not parse_obj.folder_name else parse_obj.folder_name

    for dfs in (parse_obj.dfs,
               parse_obj.skipped_dfs):
        save(list_of_dfs=dfs,
             year=year,
             folder=f"../src/{year}/{folder_name}")